# DeepSeek 7B für Deutsch trainieren

Dieses Notebook führt folgende Schritte durch:
1. DeepSeek 7B herunterladen
2. Deutsche Trainingsdaten vorbereiten
3. LoRA-Training für Deutsch durchführen
4. Chinesische Token entfernen
5. Modell quantisieren und exportieren

**Hinweis:** Dieses Notebook ist für die Ausführung auf Google Colab Pro mit A100 GPU konzipiert.

## 1. Umgebung einrichten und GPU prüfen

Zuerst stellen wir sicher, dass wir Zugriff auf eine GPU haben und richten die Umgebung ein.

In [ ]:
# GPU prüfen
!nvidia-smi

In [ ]:
# Notwendige Pakete installieren
!pip install -q transformers==4.36.2 datasets==2.15.0 peft==0.6.0 accelerate==0.24.1 bitsandbytes==0.41.1 torch==2.1.0 sentencepiece==0.1.99 gradio==3.50.2 wandb==0.15.12

import os
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
import bitsandbytes as bnb
from datasets import load_dataset
import gc

# Überprüfen, ob wir auf einer GPU laufen
print(f"CUDA verfügbar: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Modell: {torch.cuda.get_device_name(0)}")
    print(f"GPU Speicher: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. DeepSeek 7B Modell herunterladen

Wir laden das DeepSeek-Coder 7B Basismodell, welches wir später für Deutsch optimieren werden.

In [ ]:
# Modellname festlegen
model_id = "deepseek-ai/deepseek-coder-7b-base"

# Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Basiskonfigurationen für das Modell
# Wir laden das Modell im 4-bit Quantisierungsmodus, um Speicherplatz zu sparen
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, 
    device_map="auto",
    trust_remote_code=True,
    quantization_config=transformers.BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
)

# Wir überprüfen das Modell-Vokabular
print(f"Vokabulargröße: {len(tokenizer)}")
# Anzeigen einiger Beispieltokens
print("Beispieltokens:")
for i in range(10):
    print(f"Token {i}: {tokenizer.decode([i])}")

## 3. Deutsche Trainingsdaten vorbereiten

Wir benötigen deutsche Texte für das Training. Dafür nutzen wir den Oscar Corpus, der hochwertige deutsche Textdaten enthält.

In [ ]:
# Laden eines deutschen Teildatensatzes aus dem Oscar Corpus
from datasets import load_dataset

# Wir laden den deutschen Teil des Oscar Corpus
# In der Produktion würden Sie mehr Daten verwenden
dataset = load_dataset("oscar", "unshuffled_deduplicated_de", split="train", streaming=True)
dataset = dataset.take(5000)  # Wir nehmen nur 5000 Beispiele für das Demo

# Daten in eine Liste umwandeln
german_texts = [item["text"] for item in dataset]

# Kurze Prüfung der Daten
print(f"Anzahl der Texte: {len(german_texts)}")
print("Beispieltext:")
print(german_texts[0][:500] + "...")

In [ ]:
# Tokenisierung der Daten für das Training
def tokenize_function(examples):
    # Wir fügen spezielle Tokens hinzu, um das Format an das DeepSeek-Modell anzupassen
    formatted_texts = [f"<｜begin▁of▁sentence｜>{text}<｜end▁of▁sentence｜>" for text in examples]
    return tokenizer(formatted_texts, padding="max_length", truncation=True, max_length=512)

# Tokenisierung der deutschen Texte
tokenized_german_texts = tokenize_function(german_texts)

# Erstellen eines HuggingFace Datasets für das Training
import numpy as np
from datasets import Dataset

train_dataset = Dataset.from_dict({
    "input_ids": tokenized_german_texts["input_ids"],
    "attention_mask": tokenized_german_texts["attention_mask"]
})

# Datensatz in Trainings- und Validierungsdaten aufteilen
train_val_split = train_dataset.train_test_split(test_size=0.1)
train_data = train_val_split["train"]
val_data = train_val_split["test"]

print(f"Trainingsdaten: {len(train_data)} Beispiele")
print(f"Validierungsdaten: {len(val_data)} Beispiele")

## 4. LoRA-Training für Deutsch

Wir verwenden LoRA (Low-Rank Adaptation), um das Modell effizient für Deutsch zu optimieren, ohne alle Parameter neu trainieren zu müssen.

In [ ]:
# Modell für LoRA-Training vorbereiten
model = prepare_model_for_kbit_training(model)

# LoRA-Konfiguration
peft_config = LoraConfig(
    r=16,  # Rang der LoRA-Matrix
    lora_alpha=32,  # Skalierungsfaktor
    lora_dropout=0.05,  # Dropout-Rate
    bias="none",  # Keine Bias-Parameter anpassen
    task_type="CAUSAL_LM",  # Aufgabentyp: Kausales Sprachmodell
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Zielmodule für LoRA
)

# LoRA-Modell erstellen
model = get_peft_model(model, peft_config)

In [ ]:
# Trainingsargumente
training_args = transformers.TrainingArguments(
    output_dir="./deepseek-german-lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=100,
    save_total_limit=3,
    fp16=True,
    report_to="none",  # Sie können "wandb" verwenden, wenn Sie Weights & Biases nutzen möchten
    evaluation_strategy="steps",
    eval_steps=100,
)

# Trainer initialisieren
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

# Training starten
trainer.train()

# Modell speichern
trainer.save_model("./deepseek-german-lora-final")

## 5. Chinesische Token entfernen

Wir analysieren das Vokabular und bereiten die Entfernung der chinesischen Token vor.

In [ ]:
# Identifizierung und Analyse der chinesischen Token im Vokabular
def is_chinese_char(c):
    """Überprüft, ob ein Zeichen chinesisch ist."""
    cp = ord(c)
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  # CJK Unified Ideographs
        (cp >= 0x3400 and cp <= 0x4DBF) or  # CJK Unified Ideographs Extension A
        (cp >= 0x20000 and cp <= 0x2A6DF) or  # CJK Unified Ideographs Extension B
        (cp >= 0x2A700 and cp <= 0x2B73F) or  # CJK Unified Ideographs Extension C
        (cp >= 0x2B740 and cp <= 0x2B81F) or  # CJK Unified Ideographs Extension D
        (cp >= 0x2B820 and cp <= 0x2CEAF) or  # CJK Unified Ideographs Extension E
        (cp >= 0xF900 and cp <= 0xFAFF) or  # CJK Compatibility Ideographs
        (cp >= 0x2F800 and cp <= 0x2FA1F)):  # CJK Compatibility Ideographs Supplement
        return True
    return False

# Analyse des Vokabulars
token_counts = {
    "chinese": 0,
    "non_chinese": 0,
    "mixed": 0
}

# Wir gehen durch die ersten 10.000 Token, um eine Übersicht zu bekommen
for i in range(min(10000, len(tokenizer))):
    token = tokenizer.decode([i])
    chinese_chars = 0
    total_chars = 0
    
    for c in token:
        if c.strip():  # Nur nicht-Leerzeichen zählen
            total_chars += 1
            if is_chinese_char(c):
                chinese_chars += 1
    
    if total_chars > 0:
        chinese_ratio = chinese_chars / total_chars
        if chinese_ratio > 0.8:
            token_counts["chinese"] += 1
        elif chinese_ratio > 0:
            token_counts["mixed"] += 1
        else:
            token_counts["non_chinese"] += 1

print("Token-Analyse:")
print(f"Chinesisch-dominierte Token: {token_counts['chinese']}")
print(f"Gemischte Token: {token_counts['mixed']}")
print(f"Nicht-chinesische Token: {token_counts['non_chinese']}")

In [ ]:
# Wir erstellen eine Liste der Token-IDs, die wir behalten möchten (nicht-chinesische und wichtige gemischte)
retained_token_ids = []
for i in range(len(tokenizer)):
    token = tokenizer.decode([i])
    chinese_chars = 0
    total_chars = 0
    
    for c in token:
        if c.strip():
            total_chars += 1
            if is_chinese_char(c):
                chinese_chars += 1
    
    # Behalte Token, die nicht überwiegend chinesisch sind oder spezielle Token
    if total_chars == 0 or chinese_chars / total_chars < 0.8 or i < 1000:  # Wichtige spezielle Token behalten
        retained_token_ids.append(i)

print(f"Token behalten: {len(retained_token_ids)} von {len(tokenizer)}")

# Hinweis: Die vollständige Entfernung der Token würde eine Neuinitialisierung des Tokenizers erfordern,
# was komplexer ist und hier nur vorbereitet wird. In der Praxis müsste man das Modell mit dem
# bereinigten Vokabular neu trainieren.

## 6. Modell quantisieren und exportieren

Wir kombinieren das Basismodell mit den LoRA-Gewichten und quantisieren es für die Produktion.

In [ ]:
# Laden des trainierten LoRA-Modells
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Basismodell laden
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# LoRA-Modell laden und mit Basismodell kombinieren
peft_model = PeftModel.from_pretrained(base_model, "./deepseek-german-lora-final")
merged_model = peft_model.merge_and_unload()

# 8-bit Quantisierungskonfiguration
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

# Modell speichern
merged_model.save_pretrained(
    "./deepseek-german-8bit",
    quantization_config=quantization_config
)

# Tokenizer speichern
tokenizer.save_pretrained("./deepseek-german-8bit")

In [ ]:
# Optional: Konvertierung in GGUF-Format für llama.cpp
# Llama.cpp klonen
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make

# Konvertierung des Modells in das GGUF-Format
!python llama.cpp/convert.py ./deepseek-german-8bit --outtype q4_0 --outfile ./deepseek-german-q4_0.gguf

# Modell auf Google Drive speichern für den Download
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/KI-Projekt/models/deepseek-german
!cp ./deepseek-german-q4_0.gguf /content/drive/MyDrive/KI-Projekt/models/deepseek-german/
!cp -r ./deepseek-german-8bit /content/drive/MyDrive/KI-Projekt/models/deepseek-german/

## 7. Modell testen

Wir testen das trainierte und quantisierte Modell mit einem deutschen Prompt.

In [ ]:
# Laden des quantisierten Modells für Tests
test_model = AutoModelForCausalLM.from_pretrained(
    "./deepseek-german-8bit",
    device_map="auto",
    trust_remote_code=True
)
test_tokenizer = AutoTokenizer.from_pretrained("./deepseek-german-8bit", trust_remote_code=True)

# Beispielprompt
prompt = "<｜begin▁of▁sentence｜>Hallo, ich bin ein deutsch sprechender KI-Assistent. Wie kann ich dir heute helfen?<｜end▁of▁sentence｜>"

# Inferenz
input_ids = test_tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generierung
with torch.no_grad():
    output = test_model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

# Ausgabe decodieren
generated_text = test_tokenizer.decode(output[0], skip_special_tokens=True)
print("Generierter Text:")
print(generated_text)